In [1]:
import functools
import sys
import geopandas as gpd
import numpy as np
import pandas as pd
import xarray as xr
from s3fs import S3FileSystem
fs = S3FileSystem()


In [2]:
mapper = fs.get_mapper('s3://carbonplan-climatetrace/v0.3/global/3000m/raster.zarr')
ds = xr.open_zarr(mapper, consolidated=True)

In [ ]:
for variable in ['emissions_from_clearing', 'emissions_from_fire']:
    df = ds[variable].to_dataframe()
    # remove any pixels that are below 1000 tons. this makes the geojson
    # much smaller and easier to work with
    subset = df[df[variable] > 1000]
    final = subset.reset_index()
    lon = final['lon']
    lat = final['lat']
    out = final[['year', variable]].rename(columns={'year': 'y', variable: 'e'})
    out['e'] = np.round((out['e']), 2)
    out['i'] = list(out.index)
    gdf = gpd.GeoDataFrame(out, geometry=gpd.points_from_xy(lon, lat))

    with fsspec.open(f's3://carbonplan-scratch/{variable}.json', mode='w') as f:
        f.write(gdf.to_json())
#     out_mapper = fs.get_mapper(f'{variable}.geojson')
#     gdf.to_file(f'{variable}.geojson', driver='GeoJSON')

working on emissions_from_fire
